In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Getting the tables of optical thickness, calculated using HITRAN values in 'water_vapor_optical_thickness.ipynb'. Also getting the table of optical mass, calculated in 'water_vapor_profile.ipynb' using GFS model values. 

In [2]:
#---Open the look-up-table made in water_vapor_optical_thickness
optical_thickness_df_07 = pd.read_pickle('tables/optical_thickness_band07')
optical_thickness_df_13 = pd.read_pickle('tables/optical_thickness_band13')
optical_thickness_df_14 = pd.read_pickle('tables/optical_thickness_band14')

#---Open the optical mass made in water_vapor_profile
optical_mass_df = pd.read_pickle('tables/optical_mass_example')

In [3]:
optical_thickness_df_07

,Optical Thickness
1000.0,5.284162e-05
975.0,4.399260e-05
950.0,4.131971e-05
925.0,3.872974e-05
900.0,7.043308e-05
850.0,5.670890e-05
800.0,4.809045e-05
750.0,4.099548e-05
700.0,2.732179e-05
650.0,1.050884e-05


Calculating expected radiance ($I_\lambda$):

$$
I_\lambda^\uparrow (\infty, \mu)= B_\lambda^\uparrow (T_\text{sfc}) e^{-\tau^*_\lambda / \mu} + \sum_{i=0}^N B_\lambda^\uparrow (z_i) \frac{d\mathfrak{T}_\lambda^\uparrow (z_i, \infty, \mu)}{dz} \Delta z_i
$$

* $B_\lambda^\uparrow (T_\text{sfc})$ is the blackbody radiance from the surface temperature
* $\tau^*$ is optical thickness of atmosphere
* $\mathfrak{T}_\lambda^\uparrow (z_i, \infty, \mu)$ is transmittance upward from height level ($z_i$) to TOA ($\infty$) with viewing angle ($\mu$)

Calculating the blackbody radiance ($B_\lambda^\uparrow$):

$$
B_\lambda (T) = \frac{2hc^2}{\lambda^5 [e^{hc/k\lambda T} -1]}
$$

* $h$ is the Planck constant, $6.626 \times 10^{-34} \text{ J s}$
* $c$ is the speed of light, $3 \times 10^{8} \text{ m s}^{-1}$
* $k$ is the Boltzmann constant, $1.380 \times 10^{-23} \text{ J K}^{-1}$

In [4]:
def blackbody_radiance(T, wl):
    h = 6.626e-34
    c = 3e8
    k = 1.380e-23
    B = (2*h*c**2)/(wl**5 * (np.exp((h*c)/(k*wl*T))-1))
    return B


Equation for expected radiance ($I$) from the surface through the TOA. Includes the total absorption by the optical thickness, but not the individual absorption/emission of each layer. 

In [5]:
T_sfc = optical_mass_df['Temperature'].loc[1000.0]

def I_sfc(T_sfc, optical_thickness_df, wl):
    mu = 1
    tau_star = optical_thickness_df['Optical Thickness'].sum()
    I_sfc = blackbody_radiance(T_sfc, wl)*np.exp(-tau_star/mu)
    return I_sfc

print("Radiance from surface (3.9um):", '{:.2e}'.format(I_sfc(T_sfc, optical_thickness_df_07, 3.9e-6)), " W m-2 sr-1")
print("Radiance from surface (11.2um):", '{:.2e}'.format(I_sfc(T_sfc, optical_thickness_df_14, 11.2e-6)), " W m-2 sr-1")


Radiance from surface (3.9um): 5.56e+05  W m-2 sr-1
Radiance from surface (11.2um): 8.20e+06  W m-2 sr-1


Expected radiance ($I$) including the absorption/emission from each atmospheric layer. This calculation currently includes the pressure of 1000 hPa, which was used as the surface. No need for the surface emission to be added again. 

In [7]:
def I_atm(optical_thickness_df, optical_mass_df, wl):

    I_levels = []
    mu = 1

    for i in range(len(optical_thickness_df)):

        #---Temperature and blackbody radiance at current level
        T = optical_mass_df['Temperature'].iloc[i]
        B = blackbody_radiance(T, wl)

        #---Optical thickness from current level up
        tau_star = optical_thickness_df['Optical Thickness'].iloc[i:].sum()
        
        #---Calculation for dz
        if i < len(optical_thickness_df)-1:
            dz = optical_thickness_df.index[i] - optical_thickness_df.index[i+1]
        
        #---Radiance from each atmospheric level
        I_level = B*np.exp(-tau_star/mu)/dz
        I_levels.append(I_level)

    #---Summing the radiance to get the total at TOA
    I_atm = np.sum(I_levels)
    return I_atm

print("Radiance from atmosphere (3.9um): ", '{:.2e}'.format(I_atm(optical_thickness_df_07, optical_mass_df, 3.9e-6)), " W m-2 sr-1")
print("Radiance from atmosphere (11.2um): ", '{:.2e}'.format(I_atm(optical_thickness_df_14, optical_mass_df, 11.2e-6)), " W m-2 sr-1")

Radiance from atmosphere (3.9um):  1.25e+05  W m-2 sr-1
Radiance from atmosphere (11.2um):  2.86e+06  W m-2 sr-1


Calculating the brightness temperature ($T_{b,\lambda}$) from the expected radiance: 

$$
T_{b,\lambda} = \frac{hc}{k\lambda \text{ ln}\left[1+\frac{2hc^2}{I \lambda^5}\right]}
$$

In [8]:
def brightness_temperature(I, wl):
    h = 6.626e-34
    c = 3e8
    k = 1.380e-23
    Tb = (h*c)/(k*wl * np.log(1 + ((2*h*c**2)/(I*wl**5))))
    return Tb

In [11]:
print("Brightness temperature from atm (3.9um): ", round(brightness_temperature(I_atm(optical_thickness_df_07, optical_mass_df, 3.9e-6), 3.9e-6)), " K")
print("Brightness temperature from atm (10.3um): ", round(brightness_temperature(I_atm(optical_thickness_df_13, optical_mass_df, 10.3e-6), 10.3e-6)), " K")
print("Brightness temperature from atm (11.2um): ", round(brightness_temperature(I_atm(optical_thickness_df_14, optical_mass_df, 11.2e-6), 11.2e-6)), " K")

print("Brightness temperature from sfc (3.9um): ", round(brightness_temperature(I_sfc(T_sfc, optical_thickness_df_07, 3.9e-6), 3.9e-6)), " K")
print("Brightness temperature from sfc (10.3um): ", round(brightness_temperature(I_sfc(T_sfc, optical_thickness_df_13, 10.3e-6), 10.3e-6)), " K")
print("Brightness temperature from sfc (11.2um): ", round(brightness_temperature(I_sfc(T_sfc, optical_thickness_df_14, 11.2e-6), 11.2e-6)), " K")

Brightness temperature from atm (3.9um):  266  K
Brightness temperature from atm (10.3um):  239  K
Brightness temperature from atm (11.2um):  235  K
Brightness temperature from sfc (3.9um):  298  K
Brightness temperature from sfc (10.3um):  295  K
Brightness temperature from sfc (11.2um):  291  K
